# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.1.0'

## Part 1 - Data Preprocessing

In [3]:
import import_ipynb
from image_dataset_import import ImagesDataset
from pathlib import Path
dataset = ImagesDataset(selection_windturbine_paths=[Path("/data/projects/windturbine-identification-sentinel/croppedTiles/us-uswtdb_selection_windturbines")], 
                             selection_no_windturbine_paths=[Path("/data/projects/windturbine-identification-sentinel/croppedTiles/selection_no-windturbines")],
                             categories_windturbine_crops=[3], categories_no_windturbine_crops=[2], image_bands=["B02", "B03", "B04", "B08"])
X, y = dataset.create_wt_identification_data()

importing Jupyter notebook from image_dataset_import.ipynb


/home/tbroethaler/.conda/envs/machine_learning/lib/python3.6/site-packages/rasterio/__init__.py:193: UserWarning: Dataset has no geotransform set.  Default transform will be applied (Affine.identity())
  s.start()


In [5]:
dataset.indices

['7',
 '10',
 '13',
 '17',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '50',
 '51',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '80',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '110',
 '113',
 '114',
 '116',
 '117',
 '118',
 '122',
 '123',
 '124',
 '125',
 '132',
 '135',
 '138',
 '140',
 '141',
 '144',
 '145',
 '146',
 '147',
 '148',
 '149',
 '152',
 '155',
 '157',
 '158',
 '160',
 '162',
 '163',
 '164',
 '165',
 '166',
 '168',
 '171',
 '174',
 '176',
 '177',
 '178',
 '181',
 '182',
 '187',
 '188',
 '189',
 '190',
 '191',
 '192',
 '195',
 '196',
 '197',
 '198',
 '201',
 '202',
 '203',
 '204',
 '205',
 '208',


### Split data into training and test data

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, indices_train, indices_test = train_test_split(X, dataset.indices, test_size = 0.2, random_state = 0)

### Preprocessing the Training set

In [73]:
train_datagen = ImageDataGenerator(rescale = 1./(2**14))
training_set = train_datagen.flow(x=X_train, y=y_train)
#training_set = train_datagen.flow_from_directory('dataset/training_set',
#                                                 target_size = (30, 30),
#                                                 batch_size = 32,
#                                                 class_mode = 'binary')

### Preprocessing the Test set

In [74]:
test_datagen = ImageDataGenerator(rescale = 1./(2**14))
test_set = test_datagen.flow(x=X_test, y=y_test)
#test_set = test_datagen.flow_from_directory('dataset/test_set',
#                                            target_size = (30, 30),
#                                            batch_size = 32,
#                                            class_mode = 'binary')

## Part 2 - Building the CNN

### Initialising the CNN

In [82]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [83]:
cnn.add(tf.keras.layers.Conv2D(filters=16, kernel_size=5, activation='relu', input_shape=[30, 30, 4]))

### Step 2 - Pooling

In [84]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [85]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=5, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a third convolutional layer

In [86]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [87]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [88]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [89]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [90]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [92]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 10)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 236 steps, validate for 59 steps
Epoch 1/10
236/236 [==============================] - 20s 86ms/step - loss: 0.1831 - accuracy: 0.9388 - val_loss: 0.0853 - val_accuracy: 0.9751
Epoch 2/10
236/236 [==============================] - 20s 85ms/step - loss: 0.0651 - accuracy: 0.9800 - val_loss: 0.0520 - val_accuracy: 0.9851
Epoch 3/10
236/236 [==============================] - 20s 86ms/step - loss: 0.0450 - accuracy: 0.9869 - val_loss: 0.0465 - val_accuracy: 0.9846
Epoch 4/10
236/236 [==============================] - 19s 82ms/step - loss: 0.0366 - accuracy: 0.9882 - val_loss: 0.0425 - val_accuracy: 0.9862
Epoch 5/10
236/236 [==============================] - 20s 85ms/step - loss: 0.0285 - accuracy: 0.9918 - val_loss: 0.0375 - val_accuracy: 0.9878
Epoch 6/10
236/236 [==============================] - 20s 84ms/step - loss: 0.0201 - accuracy: 0.9942 - val_loss: 0.0343 - val_accuracy: 0.9878
Epoch 7/10
236/236 [=======================

## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [35]:
import numpy as np
test_image = np.expand_dims(X_test[0], axis = 0)
result = cnn.predict(test_image)

array([[0.]], dtype=float32)

In [ ]:
print(prediction)

## Part 5 - Confusion Matrix

In [44]:
y_pred = cnn.predict(X_test)
y_pred = y_pred.astype(int)

print(y_pred)

[[0]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]


In [45]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 705  134]
 [   1 1044]]


0.928343949044586

## Part 6 - Data investigation

Folders in selection_no_windturbines category_2_300m_30p = 4263
Folders in uswtdb_selection_windturbines category_2_300m_30p = 3678
Folders in uswtdb_selection_windturbines category_3_300m_30p = 5158

--------------------------------------------------------------------------------------------------------

For this test in total 4263 (-1 for 0_combined preview) + 5158 (-1 for 0_combined preview) = 9419

In [55]:
X_test.shape

(1884, 30, 30, 4)

In [56]:
X_train.shape

(7535, 30, 30, 4)

In [57]:
X.shape

(9419, 30, 30, 4)